# DDSM Mask RCNN training


In [1]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
import time
import concurrent.futures
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import imgaug
from imgaug import augmenters as iaa

# Root directory of the project
ROOT_DIR = os.getcwd()
print(ROOT_DIR)
# if ROOT_DIR.endswith("samples/nucleus"):
if ROOT_DIR.endswith("mammography"):
    # Go up one level to the repo root
    ROOT_DIR = os.path.dirname(ROOT_DIR)
    print(ROOT_DIR)
    
# Import Mask RCNN
sys.path.append(ROOT_DIR)
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn import model as modellib
from mrcnn.model import log

import mammo_ddsm
import mammo

COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

%matplotlib inline 

C:\Users\Chevy\Documents\GitHub\Mammo_MaskRCNN\mammography
C:\Users\Chevy\Documents\GitHub\Mammo_MaskRCNN


C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Comment out to reload imported modules if they change
%load_ext autoreload
%autoreload 2

## Configurations

In [3]:
# Dataset directory
DATASET_DIR = os.path.join(ROOT_DIR, "datasets/mammo")

# Load dataset
augmented = True
dataset_train = mammo.MammoDataset()
dataset_train.load_mammo(DATASET_DIR, subset="mass_train_3x", augmented=augmented)
# dataset_train.load_mammo(DATASET_DIR, subset="calc_train", augmented=augmented,  json_filename="mammo_ddsm_calc_train.json")
# dataset_train.load_mammo(DATASET_DIR, subset="calc_train_3x", augmented=augmented,  json_filename="mammo_ddsm_calc_train.json")
# dataset_train.load_mammo(DATASET_DIR, subset="mass_train", augmented=False)

# Must call before using the dataset
dataset_train.prepare()

print("Image Count: {}".format(len(dataset_train.image_ids)))
print("Class Count: {}".format(dataset_train.num_classes))
for i, info in enumerate(dataset_train.class_info):
    print("{:3}. {:50}".format(i, info['name']))
    

# Load validation dataset
dataset_val = mammo.MammoDataset()
dataset_val.load_mammo(DATASET_DIR, "val", augmented=augmented)
# dataset_val.load_mammo(DATASET_DIR, "calc_val", augmented=augmented, json_filename="mammo_ddsm_calc_train.json")
dataset_val.prepare()

print("Images: {}\nClasses: {}".format(len(dataset_val.image_ids), dataset_val.class_names))




mass_train_3x
Image Count: 3933
Class Count: 2
  0. BG                                                
  1. mass                                              
val
Images: 983
Classes: ['BG', 'mass']


In [4]:
# # Dataset directory
# DATASET_DIR = os.path.join(ROOT_DIR, "datasets/mammo")

# # Load dataset
# augmented = True
# dataset_train = mammo_ddsm.MammoDataset()
# dataset_train.load_mammo(DATASET_DIR, subset="mass_train_3x", augmented=augmented, json_filename="mammo_ddsm.json")
# # dataset_train.load_mammo(DATASET_DIR, subset="calc_train", augmented=augmented,  json_filename="mammo_ddsm_calc_train.json")
# # dataset_train.load_mammo(DATASET_DIR, subset="calc_train_3x", augmented=augmented,  json_filename="mammo_ddsm_calc_train.json")
# # dataset_train.load_mammo(DATASET_DIR, subset="mass_train", augmented=False)

# # Must call before using the dataset
# dataset_train.prepare()

# print("Image Count: {}".format(len(dataset_train.image_ids)))
# print("Class Count: {}".format(dataset_train.num_classes))
# for i, info in enumerate(dataset_train.class_info):
#     print("{:3}. {:50}".format(i, info['name']))
    

# # Load validation dataset
# dataset_val = mammo_ddsm.MammoDataset()
# dataset_val.load_mammo(DATASET_DIR, "mass_val", augmented=augmented, json_filename="mammo_ddsm.json")
# # dataset_val.load_mammo(DATASET_DIR, "calc_val", augmented=augmented, json_filename="mammo_ddsm_calc_train.json")
# dataset_val.prepare()

# print("Images: {}\nClasses: {}".format(len(dataset_val.image_ids), dataset_val.class_names))




## Notebook Preferences

In [5]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [6]:
#######################################
#   Training with higher resolution   #
#      images. Max_dim = 1408         # 
#      Training set size = 4x         #   
#          Mass_train_3x              #
#           ResNet101                 #
#       Changed Augmentation          #
#     augmentation = iaa.SomeOf((2,3), [ #
#        iaa.Fliplr(0.5),             # 
#        iaa.Flipud(0.5)              #
#######################################



# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo_ddsm.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1
    
    IMAGE_MAX_DIM = 1408

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=5, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1408
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1408 1408    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Conv2D)
bn4n_branch2c          (BatchNorm)
res4o_branch2a         (Conv2D)
bn4o_branch2a          (Batch

In [6]:
#######################################
#   Training with higher resolution   #
#      images. Max_dim = 1408         # 
#      Training set size = 4x         #   
#          Mass_train_3x              #
#           ResNet101                 #
#       Changed Augmentation          #
#     augmentation = iaa.SomeOf((2,3), [ #
#        iaa.Fliplr(0.5),             # 
#        iaa.Flipud(0.5)              #
#######################################



# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo_ddsm.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1
    
    IMAGE_MAX_DIM = 1408

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=8, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1408
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1408 1408    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


res4f_branch2b         (Conv2D)
bn4f_branch2b          (BatchNorm)
res4f_branch2c         (Conv2D)
bn4f_branch2c          (BatchNorm)
res4g_branch2a         (Conv2D)
bn4g_branch2a          (BatchNorm)
res4g_branch2b         (Conv2D)
bn4g_branch2b          (BatchNorm)
res4g_branch2c         (Conv2D)
bn4g_branch2c          (BatchNorm)
res4h_branch2a         (Conv2D)
bn4h_branch2a          (BatchNorm)
res4h_branch2b         (Conv2D)
bn4h_branch2b          (BatchNorm)
res4h_branch2c         (Conv2D)
bn4h_branch2c          (BatchNorm)
res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchN

In [ ]:
#######################################
#   Training with higher resolution   #
#      images. Max_dim = 1408         # 
#      Training set size = 4x         #   
#          Mass_train_3x              #
#           ResNet101                 #
#       Changed Augmentation          #
#     augmentation = iaa.SomeOf((2,3), [ #
#        iaa.Fliplr(0.5),             # 
#        iaa.Flipud(0.5)              #
#######################################



# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo_ddsm.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1
    
    IMAGE_MAX_DIM = 1408

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=10, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1408
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1408 1408    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



bn4h_branch2a          (BatchNorm)
res4h_branch2b         (Conv2D)
bn4h_branch2b          (BatchNorm)
res4h_branch2c         (Conv2D)
bn4h_branch2c          (BatchNorm)
res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Co

Epoch 10/10
 385/3933 [=>............................] - ETA: 1:17:32 - loss: 0.6762 - rpn_class_loss: 0.0031 - rpn_bbox_loss: 0.1255 - mrcnn_class_loss: 0.0399 - mrcnn_bbox_loss: 0.1873 - mrcnn_mask_loss: 0.3204

In [ ]:
#######################################
#   Training with higher resolution   #
#      images. Max_dim = 1024         # 
#      Training set size = 4x         #   
#          Mass_train                 #
#         ResNet50                    #
#######################################



# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo_ddsm.MammoConfig):
    BACKBONE = "resnet50"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2
    
    IMAGE_MAX_DIM = 1024

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=12, layers='all')



Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTAN

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn5b_branch2b          (BatchNorm)
res5b_branch2c         (Conv2D)
bn5b_branch2c          (BatchNorm)
res5c_branch2a         (Conv2D)
bn5c_branch2a          (BatchNorm)
res5c_branch2b         (Conv2D)
bn5c_branch2b          (BatchNorm)
res5c_branch2c         (Conv2D)
bn5c_branch2c          (BatchNorm)
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (T

In [6]:
#######################################
#   Training with higher resolution   #
#      images. Max_dim = 1024         # 
#      Training set size = 4x         # 
#   Augmentation = FlipLR and FlipUD  #
#             Calc_train              #
#######################################



# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo_ddsm.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2
    
    IMAGE_MAX_DIM = 1024

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=15, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



bn4g_branch2c          (BatchNorm)
res4h_branch2a         (Conv2D)
bn4h_branch2a          (BatchNorm)
res4h_branch2b         (Conv2D)
bn4h_branch2b          (BatchNorm)
res4h_branch2c         (Conv2D)
bn4h_branch2c          (BatchNorm)
res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Co

In [12]:
#######################################
#   Training with higher resolution   #
#      images. Max_dim = 1408         # 
#      Training set size = 4x         # 
#   Augmentation = FlipLR and FlipUD  #
#             Calc_train              #
#######################################



# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo_ddsm.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2
    
    IMAGE_MAX_DIM = 1024

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=20, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Con

In [5]:
#######################################
#   Training with higher resolution   #
#      images. Max_dim = 1408         # 
#      Training set size = 4x         # 
#   Augmentation = FlipLR and FlipUD  #
#          With Pathology             #
#######################################



# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo_ddsm.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2
    
    IMAGE_MAX_DIM = 1024

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=10, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (Batch

In [5]:
#######################################
#   Training with higher resolution   #
#      images. Max_dim = 1408         # 
#      Training set size = 4x         # 
#   Augmentation = FlipLR and FlipUD  #
#          Without Pathology          #
#######################################



# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1
    
    IMAGE_MAX_DIM = 1408

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=10, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1408
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1408 1408    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


res4g_branch2a         (Conv2D)
bn4g_branch2a          (BatchNorm)
res4g_branch2b         (Conv2D)
bn4g_branch2b          (BatchNorm)
res4g_branch2c         (Conv2D)
bn4g_branch2c          (BatchNorm)
res4h_branch2a         (Conv2D)
bn4h_branch2a          (BatchNorm)
res4h_branch2b         (Conv2D)
bn4h_branch2b          (BatchNorm)
res4h_branch2c         (Conv2D)
bn4h_branch2c          (BatchNorm)
res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchN

KeyboardInterrupt: 

In [5]:
#######################################
#   Training with higher resolution   #
#      images. Max_dim = 1408         # 
#      Training set size = 4x         # 
#       Augmentation = FlipLR         #
#          Without Pathology          #
#######################################



# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1
    
    IMAGE_MAX_DIM = 1408

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=10, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1408
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1408 1408    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4g_branch2b          (BatchNorm)
res4g_branch2c         (Conv2D)
bn4g_branch2c          (BatchNorm)
res4h_branch2a         (Conv2D)
bn4h_branch2a          (BatchNorm)
res4h_branch2b         (Conv2D)
bn4h_branch2b          (BatchNorm)
res4h_branch2c         (Conv2D)
bn4h_branch2c          (BatchNorm)
res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Con

In [ ]:
#######################################
#             22nd August             #
#   Training with higher resolution   #
#      images. Max_dim = 1408         # 
#      Training set size = 1x         #   
#######################################



# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1
    
    IMAGE_MAX_DIM = 1408

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=30, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1408
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1408 1408    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4h_branch2b          (BatchNorm)
res4h_branch2c         (Conv2D)
bn4h_branch2c          (BatchNorm)
res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Con

Epoch 10/30
983/983 [==============================] - 1450s 1s/step - loss: 0.7980 - rpn_class_loss: 0.0041 - rpn_bbox_loss: 0.1525 - mrcnn_class_loss: 0.0603 - mrcnn_bbox_loss: 0.2239 - mrcnn_mask_loss: 0.3571 - val_loss: 1.0811 - val_rpn_class_loss: 0.0048 - val_rpn_bbox_loss: 0.2960 - val_mrcnn_class_loss: 0.0708 - val_mrcnn_bbox_loss: 0.3182 - val_mrcnn_mask_loss: 0.3913
Epoch 11/30
983/983 [==============================] - 1457s 1s/step - loss: 0.7518 - rpn_class_loss: 0.0038 - rpn_bbox_loss: 0.1327 - mrcnn_class_loss: 0.0559 - mrcnn_bbox_loss: 0.2121 - mrcnn_mask_loss: 0.3474 - val_loss: 1.0895 - val_rpn_class_loss: 0.0048 - val_rpn_bbox_loss: 0.2948 - val_mrcnn_class_loss: 0.0663 - val_mrcnn_bbox_loss: 0.3219 - val_mrcnn_mask_loss: 0.4017
Epoch 12/30
983/983 [==============================] - 1460s 1s/step - loss: 0.7193 - rpn_class_loss: 0.0038 - rpn_bbox_loss: 0.1197 - mrcnn_class_loss: 0.0526 - mrcnn_bbox_loss: 0.1967 - mrcnn_mask_loss: 0.3464 - val_loss: 1.0903 - val_rpn_c

In [7]:
#######################################
#   Training with higher resolution   #
#      images. Max_dim = 1408         # 
#      Training set size = 4x         #   
#######################################



# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1
    
    IMAGE_MAX_DIM = 1408

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=10, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1408
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1408 1408    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


res4g_branch2a         (Conv2D)
bn4g_branch2a          (BatchNorm)
res4g_branch2b         (Conv2D)
bn4g_branch2b          (BatchNorm)
res4g_branch2c         (Conv2D)
bn4g_branch2c          (BatchNorm)
res4h_branch2a         (Conv2D)
bn4h_branch2a          (BatchNorm)
res4h_branch2b         (Conv2D)
bn4h_branch2b          (BatchNorm)
res4h_branch2c         (Conv2D)
bn4h_branch2c          (BatchNorm)
res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchN

KeyboardInterrupt: 

In [6]:
#######################################
#   Training with higher resolution   #
#      images. Max_dim = 1408         # 
#      Training set size = 4x         #   
#######################################



# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1
    
    IMAGE_MAX_DIM = 1408

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=10, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1408
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1408 1408    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchN

In [6]:
#######################################
#   Training with higher resolution   #
#      images. Max_dim = 1408         # 
#      Training set size = 1x         #   
#######################################



# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1
    
    IMAGE_MAX_DIM = 1408

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=20, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1408
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1408 1408    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_mask_deconv      (TimeDistributed)
mrcnn_class_logits     (TimeDistributed)
mrcnn_mask             (TimeDistributed)
Epoch 1/20
983/983 [==============================] - 1344s 1s/step - loss: 1.3146 - rpn_class_loss: 0.0275 - rpn_bbox_loss: 0.6835 - mrcnn_class_loss: 0.0280 - mrcnn_bbox_loss: 0.360

In [4]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=20, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Con

Epoch 11/20
491/491 [==============================] - 756s 2s/step - loss: 0.7660 - rpn_class_loss: 0.0045 - rpn_bbox_loss: 0.1302 - mrcnn_class_loss: 0.0614 - mrcnn_bbox_loss: 0.2188 - mrcnn_mask_loss: 0.3510 - val_loss: 1.0553 - val_rpn_class_loss: 0.0045 - val_rpn_bbox_loss: 0.2809 - val_mrcnn_class_loss: 0.0607 - val_mrcnn_bbox_loss: 0.3197 - val_mrcnn_mask_loss: 0.3894
Epoch 12/20
491/491 [==============================] - 760s 2s/step - loss: 0.7385 - rpn_class_loss: 0.0042 - rpn_bbox_loss: 0.1216 - mrcnn_class_loss: 0.0597 - mrcnn_bbox_loss: 0.2054 - mrcnn_mask_loss: 0.3476 - val_loss: 0.9921 - val_rpn_class_loss: 0.0053 - val_rpn_bbox_loss: 0.2389 - val_mrcnn_class_loss: 0.0743 - val_mrcnn_bbox_loss: 0.2969 - val_mrcnn_mask_loss: 0.3767
Epoch 13/20
491/491 [==============================] - 757s 2s/step - loss: 0.7016 - rpn_class_loss: 0.0041 - rpn_bbox_loss: 0.1061 - mrcnn_class_loss: 0.0553 - mrcnn_bbox_loss: 0.1939 - mrcnn_mask_loss: 0.3423 - val_loss: 1.0244 - val_rpn_clas

## 17 Aug
NMS_THRESHOLD = 0.8
Training set = 3x dataset

In [5]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.8
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=20, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Co

In [ ]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=20, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



bn4g_branch2c          (BatchNorm)
res4h_branch2a         (Conv2D)
bn4h_branch2a          (BatchNorm)
res4h_branch2b         (Conv2D)
bn4h_branch2b          (BatchNorm)
res4h_branch2c         (Conv2D)
bn4h_branch2c          (BatchNorm)
res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Co

##  15 Aug
3x dataset
Changes: RPN_NMS_THRESHOLD = 0.5 (From 0.9)

In [ ]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.5
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=15, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4g_branch2b          (BatchNorm)
res4g_branch2c         (Conv2D)
bn4g_branch2c          (BatchNorm)
res4h_branch2a         (Conv2D)
bn4h_branch2a          (BatchNorm)
res4h_branch2b         (Conv2D)
bn4h_branch2b          (BatchNorm)
res4h_branch2c         (Conv2D)
bn4h_branch2c          (BatchNorm)
res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Con

##  14 Aug
3x dataset
Changes: RPN_NMS_THRESHOLD = 0.3 (From 0.9)

In [ ]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.3
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=15, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Con

## 13 Aug
3x dataset
Changes: RPN_NMS_THRESHOLD = 0.1 (From 0.9)

In [5]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.1
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=10, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Co

## 11 Aug

In [5]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1500
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=30, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchN

KeyboardInterrupt: 

## 10 Aug
3x dataset
Changes: RPN_TRAIN_ANCHORS = 1024 (instead of 512 or 256)

In [ ]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 1024
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=30, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Con

## 8 August 
Training with 3x dataset and  RPN_TRAIN_ANCHORS_PER_IMAGE set to 512 (from 256)

In [ ]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=30, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4h_branch2c          (BatchNorm)
res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Con

1966/1966 [==============================] - 2761s 1s/step - loss: 0.8705 - rpn_class_loss: 0.0033 - rpn_bbox_loss: 0.1300 - mrcnn_class_loss: 0.2224 - mrcnn_bbox_loss: 0.1911 - mrcnn_mask_loss: 0.3238 - val_loss: 1.1865 - val_rpn_class_loss: 0.0045 - val_rpn_bbox_loss: 0.2777 - val_mrcnn_class_loss: 0.2668 - val_mrcnn_bbox_loss: 0.2751 - val_mrcnn_mask_loss: 0.3624
Epoch 11/30
1966/1966 [==============================] - 2789s 1s/step - loss: 0.8065 - rpn_class_loss: 0.0030 - rpn_bbox_loss: 0.1160 - mrcnn_class_loss: 0.2024 - mrcnn_bbox_loss: 0.1716 - mrcnn_mask_loss: 0.3136 - val_loss: 1.2176 - val_rpn_class_loss: 0.0047 - val_rpn_bbox_loss: 0.2586 - val_mrcnn_class_loss: 0.3022 - val_mrcnn_bbox_loss: 0.2871 - val_mrcnn_mask_loss: 0.3651
Epoch 12/30
1966/1966 [==============================] - 2823s 1s/step - loss: 0.7571 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.1049 - mrcnn_class_loss: 0.1895 - mrcnn_bbox_loss: 0.1561 - mrcnn_mask_loss: 0.3038 - val_loss: 1.2287 - val_rpn_class_l

## RPN anchor ratio changed

In [5]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    RPN_ANCHOR_RATIOS = [0.25, 0.5, 1]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 2000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=30, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



bn4h_branch2c          (BatchNorm)
res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Co

1966/1966 [==============================] - 3476s 2s/step - loss: 0.8008 - rpn_class_loss: 0.0037 - rpn_bbox_loss: 0.1298 - mrcnn_class_loss: 0.1669 - mrcnn_bbox_loss: 0.1814 - mrcnn_mask_loss: 0.3190 - val_loss: 1.1115 - val_rpn_class_loss: 0.0055 - val_rpn_bbox_loss: 0.2451 - val_mrcnn_class_loss: 0.2283 - val_mrcnn_bbox_loss: 0.2775 - val_mrcnn_mask_loss: 0.3551
Epoch 11/30
1966/1966 [==============================] - 3505s 2s/step - loss: 0.7395 - rpn_class_loss: 0.0034 - rpn_bbox_loss: 0.1132 - mrcnn_class_loss: 0.1490 - mrcnn_bbox_loss: 0.1638 - mrcnn_mask_loss: 0.3100 - val_loss: 1.1217 - val_rpn_class_loss: 0.0061 - val_rpn_bbox_loss: 0.2935 - val_mrcnn_class_loss: 0.1967 - val_mrcnn_bbox_loss: 0.2681 - val_mrcnn_mask_loss: 0.3573
Epoch 12/30
1966/1966 [==============================] - 3539s 2s/step - loss: 0.6906 - rpn_class_loss: 0.0033 - rpn_bbox_loss: 0.1010 - mrcnn_class_loss: 0.1366 - mrcnn_bbox_loss: 0.1480 - mrcnn_mask_loss: 0.3017 - val_loss: 1.1710 - val_rpn_class_l

KeyboardInterrupt: 

## RPN_ANCHOR_SCALE changed to (32, 64, 128, 256, 512)

In [6]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 2000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=30, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchN

KeyboardInterrupt: 

In [ ]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 2000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=30, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Con

## Monday: 30th July

In [5]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    a
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 2000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=30, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4e_branch2c          (BatchNorm)
res4f_branch2a         (Conv2D)
bn4f_branch2a          (BatchNorm)
res4f_branch2b         (Conv2D)
bn4f_branch2b          (BatchNorm)
res4f_branch2c         (Conv2D)
bn4f_branch2c          (BatchNorm)
res4g_branch2a         (Conv2D)
bn4g_branch2a          (BatchNorm)
res4g_branch2b         (Conv2D)
bn4g_branch2b          (BatchNorm)
res4g_branch2c         (Conv2D)
bn4g_branch2c          (BatchNorm)
res4h_branch2a         (Conv2D)
bn4h_branch2a          (BatchNorm)
res4h_branch2b         (Conv2D)
bn4h_branch2b          (BatchNorm)
res4h_branch2c         (Conv2D)
bn4h_branch2c          (BatchNorm)
res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Con

KeyboardInterrupt: 

In [5]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
#     STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1500
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=40, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


res4i_branch2a         (Conv2D)
bn4i_branch2a          (BatchNorm)
res4i_branch2b         (Conv2D)
bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchN

KeyboardInterrupt: 

In [ ]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1500
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=50, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Conv2D)
bn4n_branch2c          (Batch

Epoch 11/50
368/368 [==============================] - 579s 2s/step - loss: 0.9454 - rpn_class_loss: 0.0044 - rpn_bbox_loss: 0.1755 - mrcnn_class_loss: 0.1145 - mrcnn_bbox_loss: 0.2597 - mrcnn_mask_loss: 0.3913 - val_loss: 1.1219 - val_rpn_class_loss: 0.0046 - val_rpn_bbox_loss: 0.2983 - val_mrcnn_class_loss: 0.1097 - val_mrcnn_bbox_loss: 0.3137 - val_mrcnn_mask_loss: 0.3957
Epoch 12/50
368/368 [==============================] - 580s 2s/step - loss: 0.8598 - rpn_class_loss: 0.0039 - rpn_bbox_loss: 0.1577 - mrcnn_class_loss: 0.1088 - mrcnn_bbox_loss: 0.2267 - mrcnn_mask_loss: 0.3627 - val_loss: 1.1414 - val_rpn_class_loss: 0.0039 - val_rpn_bbox_loss: 0.2800 - val_mrcnn_class_loss: 0.1529 - val_mrcnn_bbox_loss: 0.3184 - val_mrcnn_mask_loss: 0.3862
Epoch 13/50
368/368 [==============================] - 582s 2s/step - loss: 0.8323 - rpn_class_loss: 0.0037 - rpn_bbox_loss: 0.1226 - mrcnn_class_loss: 0.1182 - mrcnn_bbox_loss: 0.2226 - mrcnn_mask_loss: 0.3651 - val_loss: 1.1674 - val_rpn_clas

In [6]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet50"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1500
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.75
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=50, layers='all')



Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTAN

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn5b_branch2c          (BatchNorm)
res5c_branch2a         (Conv2D)
bn5c_branch2a          (BatchNorm)
res5c_branch2b         (Conv2D)
bn5c_branch2b          (BatchNorm)
res5c_branch2c         (Conv2D)
bn5c_branch2c          (BatchNorm)
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_clas

Epoch 41/50
737/737 [==============================] - 740s 1s/step - loss: 0.2587 - rpn_class_loss: 9.7257e-04 - rpn_bbox_loss: 0.0266 - mrcnn_class_loss: 0.0143 - mrcnn_bbox_loss: 0.0378 - mrcnn_mask_loss: 0.1790 - val_loss: 1.5012 - val_rpn_class_loss: 0.0028 - val_rpn_bbox_loss: 0.3417 - val_mrcnn_class_loss: 0.1709 - val_mrcnn_bbox_loss: 0.3319 - val_mrcnn_mask_loss: 0.6539
Epoch 42/50
737/737 [==============================] - 745s 1s/step - loss: 0.2561 - rpn_class_loss: 9.6472e-04 - rpn_bbox_loss: 0.0260 - mrcnn_class_loss: 0.0131 - mrcnn_bbox_loss: 0.0380 - mrcnn_mask_loss: 0.1780 - val_loss: 1.4499 - val_rpn_class_loss: 0.0025 - val_rpn_bbox_loss: 0.3431 - val_mrcnn_class_loss: 0.1381 - val_mrcnn_bbox_loss: 0.3550 - val_mrcnn_mask_loss: 0.6112
Epoch 43/50
737/737 [==============================] - 740s 1s/step - loss: 0.2473 - rpn_class_loss: 9.0330e-04 - rpn_bbox_loss: 0.0245 - mrcnn_class_loss: 0.0120 - mrcnn_bbox_loss: 0.0368 - mrcnn_mask_loss: 0.1731 - val_loss: 1.5999 - 

KeyboardInterrupt: 

In [5]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet50"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.75
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=50, layers='all')



Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTAN

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



bn5c_branch2c          (BatchNorm)
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_mask_deconv      (TimeDistributed)
mrcnn

Epoch 20/50
737/737 [==============================] - 581s 789ms/step - loss: 0.4850 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.0550 - mrcnn_class_loss: 0.0378 - mrcnn_bbox_loss: 0.1041 - mrcnn_mask_loss: 0.2853 - val_loss: 1.1615 - val_rpn_class_loss: 0.0040 - val_rpn_bbox_loss: 0.3894 - val_mrcnn_class_loss: 0.0591 - val_mrcnn_bbox_loss: 0.3211 - val_mrcnn_mask_loss: 0.3878
Epoch 21/50
737/737 [==============================] - 582s 790ms/step - loss: 0.4607 - rpn_class_loss: 0.0025 - rpn_bbox_loss: 0.0500 - mrcnn_class_loss: 0.0360 - mrcnn_bbox_loss: 0.0971 - mrcnn_mask_loss: 0.2751 - val_loss: 1.2480 - val_rpn_class_loss: 0.0044 - val_rpn_bbox_loss: 0.3695 - val_mrcnn_class_loss: 0.1012 - val_mrcnn_bbox_loss: 0.3366 - val_mrcnn_mask_loss: 0.4363
Epoch 22/50
737/737 [==============================] - 580s 788ms/step - loss: 0.4443 - rpn_class_loss: 0.0024 - rpn_bbox_loss: 0.0490 - mrcnn_class_loss: 0.0364 - mrcnn_bbox_loss: 0.0887 - mrcnn_mask_loss: 0.2679 - val_loss: 1.3060 - val

KeyboardInterrupt: 

In [7]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet50"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.75
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=50, layers='all')



Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTAN

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_mask_deconv      (TimeDistributed)
mrcnn_class_logits     (TimeDistributed)
mrcnn_mask             (TimeDistributed)
Epoch 1/50
737/737 [==============================] - 

737/737 [==============================] - 563s 764ms/step - loss: 0.2579 - rpn_class_loss: 0.0012 - rpn_bbox_loss: 0.0265 - mrcnn_class_loss: 0.0123 - mrcnn_bbox_loss: 0.0380 - mrcnn_mask_loss: 0.1800 - val_loss: 1.5337 - val_rpn_class_loss: 0.0050 - val_rpn_bbox_loss: 0.3608 - val_mrcnn_class_loss: 0.1441 - val_mrcnn_bbox_loss: 0.3438 - val_mrcnn_mask_loss: 0.6801
Epoch 42/50
737/737 [==============================] - 559s 759ms/step - loss: 0.2611 - rpn_class_loss: 0.0012 - rpn_bbox_loss: 0.0272 - mrcnn_class_loss: 0.0133 - mrcnn_bbox_loss: 0.0404 - mrcnn_mask_loss: 0.1790 - val_loss: 1.6007 - val_rpn_class_loss: 0.0048 - val_rpn_bbox_loss: 0.3696 - val_mrcnn_class_loss: 0.1820 - val_mrcnn_bbox_loss: 0.3375 - val_mrcnn_mask_loss: 0.7068
Epoch 43/50
737/737 [==============================] - 562s 762ms/step - loss: 0.2537 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0265 - mrcnn_class_loss: 0.0132 - mrcnn_bbox_loss: 0.0375 - mrcnn_mask_loss: 0.1754 - val_loss: 1.5753 - val_rpn_class_l

KeyboardInterrupt: 

In [6]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet50"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=50, layers='all')



Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTAN

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_mask_deconv      (TimeDistributed)
mrcnn_class_logits     (TimeDistributed)
mrcnn_mask             (TimeDist

Epoch 20/50
1402/1402 [==============================] - 1015s 724ms/step - loss: 0.7866 - rpn_class_loss: 0.0056 - rpn_bbox_loss: 0.0975 - mrcnn_class_loss: 0.1242 - mrcnn_bbox_loss: 0.1906 - mrcnn_mask_loss: 0.3687 - val_loss: 1.5945 - val_rpn_class_loss: 0.0081 - val_rpn_bbox_loss: 0.5070 - val_mrcnn_class_loss: 0.1909 - val_mrcnn_bbox_loss: 0.4324 - val_mrcnn_mask_loss: 0.4561
Epoch 21/50
1402/1402 [==============================] - 1017s 725ms/step - loss: 0.7273 - rpn_class_loss: 0.0052 - rpn_bbox_loss: 0.0832 - mrcnn_class_loss: 0.1197 - mrcnn_bbox_loss: 0.1651 - mrcnn_mask_loss: 0.3541 - val_loss: 1.6697 - val_rpn_class_loss: 0.0078 - val_rpn_bbox_loss: 0.5848 - val_mrcnn_class_loss: 0.1837 - val_mrcnn_bbox_loss: 0.4406 - val_mrcnn_mask_loss: 0.4528
Epoch 22/50
1402/1402 [==============================] - 1016s 725ms/step - loss: 0.7255 - rpn_class_loss: 0.0051 - rpn_bbox_loss: 0.0857 - mrcnn_class_loss: 0.1159 - mrcnn_bbox_loss: 0.1644 - mrcnn_mask_loss: 0.3543 - val_loss: 1.6

KeyboardInterrupt: 

In [6]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
    BACKBONE = "resnet50"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=80, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Co

1402/1402 [==============================] - 1217s 868ms/step - loss: 0.5654 - rpn_class_loss: 0.0039 - rpn_bbox_loss: 0.0633 - mrcnn_class_loss: 0.0839 - mrcnn_bbox_loss: 0.1102 - mrcnn_mask_loss: 0.3041 - val_loss: 1.9384 - val_rpn_class_loss: 0.0087 - val_rpn_bbox_loss: 0.6428 - val_mrcnn_class_loss: 0.2934 - val_mrcnn_bbox_loss: 0.4662 - val_mrcnn_mask_loss: 0.5274
Epoch 32/80
1402/1402 [==============================] - 1215s 867ms/step - loss: 0.5547 - rpn_class_loss: 0.0039 - rpn_bbox_loss: 0.0593 - mrcnn_class_loss: 0.0779 - mrcnn_bbox_loss: 0.1109 - mrcnn_mask_loss: 0.3026 - val_loss: 1.9153 - val_rpn_class_loss: 0.0089 - val_rpn_bbox_loss: 0.5752 - val_mrcnn_class_loss: 0.3209 - val_mrcnn_bbox_loss: 0.4531 - val_mrcnn_mask_loss: 0.5572
Epoch 33/80
1402/1402 [==============================] - 1216s 867ms/step - loss: 0.5249 - rpn_class_loss: 0.0036 - rpn_bbox_loss: 0.0565 - mrcnn_class_loss: 0.0746 - mrcnn_bbox_loss: 0.1010 - mrcnn_mask_loss: 0.2893 - val_loss: 2.1208 - val_rp

1402/1402 [==============================] - 1205s 859ms/step - loss: 0.2264 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0210 - mrcnn_class_loss: 0.0225 - mrcnn_bbox_loss: 0.0305 - mrcnn_mask_loss: 0.1512 - val_loss: 2.7970 - val_rpn_class_loss: 0.0154 - val_rpn_bbox_loss: 0.6355 - val_mrcnn_class_loss: 0.6779 - val_mrcnn_bbox_loss: 0.4563 - val_mrcnn_mask_loss: 1.0118
Epoch 74/80
1402/1402 [==============================] - 1202s 857ms/step - loss: 0.2260 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0207 - mrcnn_class_loss: 0.0234 - mrcnn_bbox_loss: 0.0296 - mrcnn_mask_loss: 0.1512 - val_loss: 2.8190 - val_rpn_class_loss: 0.0164 - val_rpn_bbox_loss: 0.6178 - val_mrcnn_class_loss: 0.8066 - val_mrcnn_bbox_loss: 0.4472 - val_mrcnn_mask_loss: 0.9309
Epoch 75/80
1402/1402 [==============================] - 1202s 857ms/step - loss: 0.2288 - rpn_class_loss: 0.0010 - rpn_bbox_loss: 0.0208 - mrcnn_class_loss: 0.0249 - mrcnn_bbox_loss: 0.0303 - mrcnn_mask_loss: 0.1518 - val_loss: 2.7882 - val_rp

In [7]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=60, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Con

1402/1402 [==============================] - 1240s 884ms/step - loss: 0.5575 - rpn_class_loss: 0.0027 - rpn_bbox_loss: 0.0624 - mrcnn_class_loss: 0.0800 - mrcnn_bbox_loss: 0.1128 - mrcnn_mask_loss: 0.2995 - val_loss: 1.8726 - val_rpn_class_loss: 0.0050 - val_rpn_bbox_loss: 0.6627 - val_mrcnn_class_loss: 0.2723 - val_mrcnn_bbox_loss: 0.4478 - val_mrcnn_mask_loss: 0.4847
Epoch 32/60
1402/1402 [==============================] - 1238s 883ms/step - loss: 0.5432 - rpn_class_loss: 0.0026 - rpn_bbox_loss: 0.0599 - mrcnn_class_loss: 0.0761 - mrcnn_bbox_loss: 0.1064 - mrcnn_mask_loss: 0.2981 - val_loss: 1.8566 - val_rpn_class_loss: 0.0051 - val_rpn_bbox_loss: 0.6110 - val_mrcnn_class_loss: 0.2753 - val_mrcnn_bbox_loss: 0.4495 - val_mrcnn_mask_loss: 0.5156
Epoch 33/60
1402/1402 [==============================] - 1241s 885ms/step - loss: 0.5213 - rpn_class_loss: 0.0024 - rpn_bbox_loss: 0.0557 - mrcnn_class_loss: 0.0707 - mrcnn_bbox_loss: 0.1038 - mrcnn_mask_loss: 0.2887 - val_loss: 2.0101 - val_rp

## Train #3
### With 2000 POST_NMS_ROIS_TRAINING and 2500 POST_NMS_ROIS_INFERENCE

In [5]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 2000
    POST_NMS_ROIS_INFERENCE = 2500
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=60, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Conv2D)
bn4n_branch2c          (BatchNorm)
res4o_branch2a         (Con

Epoch 11/60
984/984 [==============================] - 1173s 1s/step - loss: 0.7815 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.1146 - mrcnn_class_loss: 0.1262 - mrcnn_bbox_loss: 0.1936 - mrcnn_mask_loss: 0.3443 - val_loss: 1.1722 - val_rpn_class_loss: 0.0031 - val_rpn_bbox_loss: 0.3903 - val_mrcnn_class_loss: 0.1228 - val_mrcnn_bbox_loss: 0.2902 - val_mrcnn_mask_loss: 0.3658
Epoch 12/60
984/984 [==============================] - 1177s 1s/step - loss: 0.7264 - rpn_class_loss: 0.0026 - rpn_bbox_loss: 0.0962 - mrcnn_class_loss: 0.1249 - mrcnn_bbox_loss: 0.1706 - mrcnn_mask_loss: 0.3321 - val_loss: 1.1936 - val_rpn_class_loss: 0.0027 - val_rpn_bbox_loss: 0.3595 - val_mrcnn_class_loss: 0.1290 - val_mrcnn_bbox_loss: 0.3180 - val_mrcnn_mask_loss: 0.3843
Epoch 13/60
984/984 [==============================] - 1178s 1s/step - loss: 0.6738 - rpn_class_loss: 0.0024 - rpn_bbox_loss: 0.0799 - mrcnn_class_loss: 0.1157 - mrcnn_bbox_loss: 0.1535 - mrcnn_mask_loss: 0.3223 - val_loss: 1.3115 - val_rpn_c

Epoch 33/60
984/984 [==============================] - 1178s 1s/step - loss: 0.2788 - rpn_class_loss: 9.7027e-04 - rpn_bbox_loss: 0.0257 - mrcnn_class_loss: 0.0380 - mrcnn_bbox_loss: 0.0361 - mrcnn_mask_loss: 0.1780 - val_loss: 2.0119 - val_rpn_class_loss: 0.0025 - val_rpn_bbox_loss: 0.3080 - val_mrcnn_class_loss: 0.6276 - val_mrcnn_bbox_loss: 0.3551 - val_mrcnn_mask_loss: 0.7187
Epoch 34/60
984/984 [==============================] - 1176s 1s/step - loss: 0.2730 - rpn_class_loss: 9.1119e-04 - rpn_bbox_loss: 0.0252 - mrcnn_class_loss: 0.0376 - mrcnn_bbox_loss: 0.0357 - mrcnn_mask_loss: 0.1735 - val_loss: 2.1726 - val_rpn_class_loss: 0.0030 - val_rpn_bbox_loss: 0.3216 - val_mrcnn_class_loss: 0.7483 - val_mrcnn_bbox_loss: 0.3666 - val_mrcnn_mask_loss: 0.7331
Epoch 35/60
984/984 [==============================] - 1177s 1s/step - loss: 0.2653 - rpn_class_loss: 9.1526e-04 - rpn_bbox_loss: 0.0260 - mrcnn_class_loss: 0.0347 - mrcnn_bbox_loss: 0.0343 - mrcnn_mask_loss: 0.1694 - val_loss: 2.3798

## Training #2 
### With proper bounding box and 60 epochs

In [6]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=60, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Con

Epoch 11/60
984/984 [==============================] - 809s 823ms/step - loss: 0.8031 - rpn_class_loss: 0.0030 - rpn_bbox_loss: 0.1218 - mrcnn_class_loss: 0.1195 - mrcnn_bbox_loss: 0.2000 - mrcnn_mask_loss: 0.3588 - val_loss: 1.1603 - val_rpn_class_loss: 0.0030 - val_rpn_bbox_loss: 0.2916 - val_mrcnn_class_loss: 0.1418 - val_mrcnn_bbox_loss: 0.3378 - val_mrcnn_mask_loss: 0.3862
Epoch 12/60
984/984 [==============================] - 810s 824ms/step - loss: 0.7393 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.0982 - mrcnn_class_loss: 0.1174 - mrcnn_bbox_loss: 0.1757 - mrcnn_mask_loss: 0.3452 - val_loss: 1.1646 - val_rpn_class_loss: 0.0029 - val_rpn_bbox_loss: 0.3261 - val_mrcnn_class_loss: 0.1456 - val_mrcnn_bbox_loss: 0.3173 - val_mrcnn_mask_loss: 0.3727
Epoch 13/60
984/984 [==============================] - 807s 820ms/step - loss: 0.7080 - rpn_class_loss: 0.0027 - rpn_bbox_loss: 0.0878 - mrcnn_class_loss: 0.1193 - mrcnn_bbox_loss: 0.1592 - mrcnn_mask_loss: 0.3391 - val_loss: 1.2166 - val

984/984 [==============================] - 831s 845ms/step - loss: 0.1840 - rpn_class_loss: 6.5612e-04 - rpn_bbox_loss: 0.0159 - mrcnn_class_loss: 0.0209 - mrcnn_bbox_loss: 0.0189 - mrcnn_mask_loss: 0.1277 - val_loss: 2.3100 - val_rpn_class_loss: 0.0033 - val_rpn_bbox_loss: 0.3501 - val_mrcnn_class_loss: 0.8233 - val_mrcnn_bbox_loss: 0.3451 - val_mrcnn_mask_loss: 0.7882
Epoch 54/60
984/984 [==============================] - 832s 846ms/step - loss: 0.1850 - rpn_class_loss: 6.8979e-04 - rpn_bbox_loss: 0.0157 - mrcnn_class_loss: 0.0220 - mrcnn_bbox_loss: 0.0192 - mrcnn_mask_loss: 0.1274 - val_loss: 2.1762 - val_rpn_class_loss: 0.0034 - val_rpn_bbox_loss: 0.3517 - val_mrcnn_class_loss: 0.6564 - val_mrcnn_bbox_loss: 0.3617 - val_mrcnn_mask_loss: 0.8030
Epoch 55/60
984/984 [==============================] - 832s 846ms/step - loss: 0.1807 - rpn_class_loss: 6.4021e-04 - rpn_bbox_loss: 0.0145 - mrcnn_class_loss: 0.0232 - mrcnn_bbox_loss: 0.0187 - mrcnn_mask_loss: 0.1236 - val_loss: 2.1490 - val

## Training #1 
### With proper bounding box and 40 epochs

In [6]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=40, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Conv2D)
bn4n_branch2c          (BatchNorm)
res4o_branch2a         (Conv2D)
bn4o_branch2a          (BatchNorm)
res4o_branch2b         (Co

Epoch 11/40
984/984 [==============================] - 1350s 1s/step - loss: 0.6850 - rpn_class_loss: 0.0027 - rpn_bbox_loss: 0.1114 - mrcnn_class_loss: 0.0289 - mrcnn_bbox_loss: 0.1996 - mrcnn_mask_loss: 0.3425 - val_loss: 1.1400 - val_rpn_class_loss: 0.0030 - val_rpn_bbox_loss: 0.3213 - val_mrcnn_class_loss: 0.0514 - val_mrcnn_bbox_loss: 0.3276 - val_mrcnn_mask_loss: 0.4367
Epoch 12/40
984/984 [==============================] - 1358s 1s/step - loss: 0.6486 - rpn_class_loss: 0.0024 - rpn_bbox_loss: 0.0993 - mrcnn_class_loss: 0.0269 - mrcnn_bbox_loss: 0.1840 - mrcnn_mask_loss: 0.3360 - val_loss: 1.0837 - val_rpn_class_loss: 0.0025 - val_rpn_bbox_loss: 0.4048 - val_mrcnn_class_loss: 0.0297 - val_mrcnn_bbox_loss: 0.2958 - val_mrcnn_mask_loss: 0.3509
Epoch 13/40
984/984 [==============================] - 1358s 1s/step - loss: 0.5991 - rpn_class_loss: 0.0023 - rpn_bbox_loss: 0.0865 - mrcnn_class_loss: 0.0244 - mrcnn_bbox_loss: 0.1646 - mrcnn_mask_loss: 0.3212 - val_loss: 1.0945 - val_rpn_c

Epoch 33/40
984/984 [==============================] - 1354s 1s/step - loss: 0.2596 - rpn_class_loss: 0.0010 - rpn_bbox_loss: 0.0281 - mrcnn_class_loss: 0.0066 - mrcnn_bbox_loss: 0.0409 - mrcnn_mask_loss: 0.1829 - val_loss: 1.3726 - val_rpn_class_loss: 0.0023 - val_rpn_bbox_loss: 0.3322 - val_mrcnn_class_loss: 0.0812 - val_mrcnn_bbox_loss: 0.3456 - val_mrcnn_mask_loss: 0.6113
Epoch 34/40
984/984 [==============================] - 1361s 1s/step - loss: 0.2577 - rpn_class_loss: 0.0010 - rpn_bbox_loss: 0.0281 - mrcnn_class_loss: 0.0067 - mrcnn_bbox_loss: 0.0406 - mrcnn_mask_loss: 0.1811 - val_loss: 1.7583 - val_rpn_class_loss: 0.0031 - val_rpn_bbox_loss: 0.3568 - val_mrcnn_class_loss: 0.1285 - val_mrcnn_bbox_loss: 0.3535 - val_mrcnn_mask_loss: 0.9164
Epoch 35/40
984/984 [==============================] - 1354s 1s/step - loss: 0.2454 - rpn_class_loss: 9.7563e-04 - rpn_bbox_loss: 0.0250 - mrcnn_class_loss: 0.0055 - mrcnn_bbox_loss: 0.0370 - mrcnn_mask_loss: 0.1769 - val_loss: 1.5053 - val_r